In [117]:
import re
import os
import httpx
from dotenv import load_dotenv
from groq import Groq

load_dotenv()  # Loads GROQ_API_KEY into os.environ

client = Groq()  # No need to pass api_key manually if .env is loaded

completion = client.chat.completions.create(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    messages=[
      {
        "role": "user",
        "content": "Hello World"
      }
    ],
    temperature=1,
    max_completion_tokens=1024,
    top_p=1,
    stream=True,
    stop=None,
)

for chunk in completion:
    print(chunk.choices[0].delta.content or "", end="")



Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?

In [118]:
class CreditCardAgent:
    def __init__(self, model_client, model="meta-llama/llama-4-scout-17b-16e-instruct"):
        self.client = model_client
        self.model = model
        self.messages = []

    def chat(self, role, content):
        self.messages.append({"role": role, "content": content})

    def generate(self):
        response = self.client.chat.completions.create(
            model=self.model,
            messages=self.messages,
            temperature=0.7
        )
        return response.choices[0].message.content

    def extract_and_update_profile(self, user_input):
        import re
        profile = {}

        income_match = re.search(r"(?:₹|Rs\.?|INR)?\s?(\d{4,7})", user_input.replace(",", ""))
        profile["income"] = int(income_match.group(1)) if income_match else 0

        profile["spend_categories"] = [w for w in ["fuel", "groceries", "dining", "travel", "shopping"] if w in user_input.lower()]

        prefs_map = {"cashback": "cashback", "lounge": "lounge access", "rewards": "reward points"}
        profile["preferences"] = [v for k, v in prefs_map.items() if k in user_input.lower()]

        score_match = re.search(r"score.*?(\d{3})", user_input.lower())
        profile["credit_score"] = int(score_match.group(1)) if score_match else 700

        user_profile.update(profile)
        return profile

    def run(self, user_input):
        self.chat("user", user_input)
        self.extract_and_update_profile(user_input)

        cards = filter_cards(user_profile)
        if not cards:
            return "Sorry, no cards match your profile."

        top_card_names = [c["name"] for c in cards[:3]]
        sim_rewards = [simulate_rewards(name, user_profile) for name in top_card_names]

        recommendations = "Top Cards:\n" + "\n".join([f"- {c['name']} ({c['reward_type']}): ₹{c['fee']}, perks: {', '.join(c['perks'])}" for c in cards])
        rewards = "\n\nRewards Estimate:\n" + "\n".join(sim_rewards)

        summary = recommendations + rewards
        self.chat("assistant", summary)
        return summary

In [ ]:
agent_prompt = """
You are an intelligent credit card advisor assistant.

Your goal is to understand the user's financial profile and recommend the best-fit Indian credit cards from a JSON database.

Follow this reasoning + action pattern:

Thought → Action → Observation → (repeat if needed) → Answer

Ask follow-up questions only when needed. Once you have the following details, proceed to filter and recommend cards:

1. Monthly income (e.g., ₹70,000)
2. Spending categories (e.g., groceries, dining, travel)
3. Preferences (e.g., cashback, lounge access)
4. Credit score (can be estimated if unknown)

Use tools:
- `filter_cards(profile)` — fetch and rank top cards
- `simulate_rewards(card_name)` — estimate annual rewards for a card
- `compare_cards(card_names)` — compare key card features

Rules:
- Avoid repeating questions once info is extracted
- Always use full card names from the database for simulation/comparison
- If a card name is vague, try to fuzzy match the best available one

Example:
User: I spend ₹75,000/month mostly on travel and dining. I want cashback and lounge benefits.

→ Thought: I have income, spend categories, and preferences.
→ Action: filter_cards
→ Observation: [Top cards listed]
→ Action: simulate_rewards on top 3 cards
→ Observation: Estimated cashback results
→ Answer: Show summary of recommendations


"""

In [120]:
import json
import difflib

with open("cards.json") as f:
    card_data = json.load(f)

def get_best_match_card_name(name):
    all_card_names = [card["name"] for card in card_data]
    match = difflib.get_close_matches(name, all_card_names, n=1, cutoff=0.6)
    return match[0] if match else None


def filter_cards(profile):
    income = profile.get("income", 0)
    score = profile.get("credit_score", 0)
    prefs = profile.get("preferences", [])
    spend = profile.get("spend_categories", [])

    def card_score(card):
        if income < card.get("min_income", 0) or (isinstance(score, int) and score < card.get("min_score", 0)):
            return -1  # reject

        reward = card.get("reward_type", "").lower()
        perks = card.get("perks", [])
        reward_cats = card.get("reward_categories", [])

        match_prefs = sum(1 for p in prefs if p in reward or p in " ".join(perks))
        match_spend = sum(1 for s in spend if s in reward_cats)

        return match_prefs * 2 + match_spend + card.get("reward_rate", 0)

    scored_cards = [(card, card_score(card)) for card in card_data]
    valid_cards = [c for c, score in scored_cards if score >= 0]
    sorted_cards = sorted(valid_cards, key=lambda c: card_score(c), reverse=True)

    return sorted_cards[:5]


def simulate_rewards(card_name, profile):
    matched_name = get_best_match_card_name(card_name)
    if not matched_name:
        return f"No match found for card: {card_name}"
    spend = profile.get("income", 0) * 0.3
    card = next((c for c in card_data if c["name"] == matched_name), None)
    matched_categories = [cat for cat in profile.get("spend_categories", []) if cat in card.get("reward_categories", [])]
    reward_rate = card.get("reward_rate", 0.01)
    multiplier = 1.5 if matched_categories else 1.0
    annual_reward = spend * reward_rate * 12 * multiplier
    return f"{matched_name}: Estimated annual reward: ₹{int(annual_reward)}"

def compare_cards(names):
    corrected_names = [get_best_match_card_name(n.strip()) for n in names]
    selected = [c for c in card_data if c["name"] in corrected_names and c["name"] is not None]
    if not selected:
        return "No matching cards found."
    result = "\n\n--- Card Comparison ---\n"
    for c in selected:
        result += f"\n**{c['name']} ({c['issuer']})**\n"
        result += f"Fee: ₹{c['fee']}, Reward: {c['reward_type']} ({c['reward_rate']}%)\n"
        result += f"Perks: {', '.join(c.get('perks', []))}\n"
        result += f"[Apply Here]({c['link']})\n"
    return result

user_profile = {}

In [121]:
agent = CreditCardAgent(client)
print(agent.run("I earn ₹75,000 and spend mostly on travel and dining. I want cashback and lounge access."))

Top Cards:
- AU Bank LIT Credit Card (cashback): ₹0, perks: customizable features, lounge access, OTT subscriptions
- Axis Bank Ace Credit Card (cashback): ₹499, perks: cashback, lounge access, dining discounts
- HDFC Millennia Credit Card (cashback): ₹1000, perks: cashback, gift vouchers
- Bank of Baroda ETERNA Credit Card (reward points): ₹2499, perks: lounge access, golf benefits, concierge services
- AU Bank Zenith Credit Card (reward points): ₹7999, perks: lounge access, bonus reward points, golf access

Rewards Estimate:
AU Bank LIT Credit Card: Estimated annual reward: ₹20250
Axis Bank Ace Credit Card: Estimated annual reward: ₹20250
HDFC Millennia Credit Card: Estimated annual reward: ₹20250


In [122]:
print(agent.run("I earn 10000 monthly, i spend on travel and living only. i want lounge access only, maybe a few cashbacks. Suggest cards."))

Sorry, no cards match your profile.


In [123]:
print(agent.run("I earn 100000 monthly, i spend on travel and living only. i want lounge access only, maybe a few cashbacks, and my credit score is 200. Suggest cards."))


Sorry, no cards match your profile.


In [124]:
print(agent.run("I earn 100000 monthly, i spend on travel and living only. i want lounge access only, maybe a few cashbacks, and my credit score is 800. Suggest cards."))



Top Cards:
- AU Bank LIT Credit Card (cashback): ₹0, perks: customizable features, lounge access, OTT subscriptions
- Axis Bank Ace Credit Card (cashback): ₹499, perks: cashback, lounge access, dining discounts
- HDFC Millennia Credit Card (cashback): ₹1000, perks: cashback, gift vouchers
- Bank of Baroda ETERNA Credit Card (reward points): ₹2499, perks: lounge access, golf benefits, concierge services
- Amazon Pay ICICI Bank Credit Card (amazon cashback): ₹0, perks: cashback

Rewards Estimate:
AU Bank LIT Credit Card: Estimated annual reward: ₹27000
Axis Bank Ace Credit Card: Estimated annual reward: ₹18000
HDFC Millennia Credit Card: Estimated annual reward: ₹27000


In [125]:
print(agent.run("Give me a list of my best credit card options in india"))


Sorry, no cards match your profile.
